## 1) Implementation of L1 regularization
In order to generalize as much as possible I've implemented a function which takes as an input the value of the loss already calculated during the training and add the regularization part 

In [1]:
def L1_reg(loss,model,l):
    L1_reg = torch.zeros(1)
    for name, param in model.named_parameters():
        if "weight" in name:
            L1_reg += torch.norm(param,1)
    return loss+l*L1_reg


## 2a) Early stop $E_{opt}$
For this purpose I've modified the `train_model` funciton in order to take as a parameter the value of alpha which will be used as a criterium, then in the function I've implemented the algorithm as follow:

    loss, _ = test_model(model,testloader,performance,loss_fn)

    E_opt = min(loss,E_opt)

        gl = 100*(loss/E_opt-1)

        if gl > alpha:

            return loss_meter.sum, performance_meter.avg, trajectory

## 2b) 

The seconnd criterium I've decided to implement is to stop when the error on testset increasen `s` successive epochs (`s` is another parameter to be passed as input):


    if loss > loss_prec:

            Up_count += 1

        else :

            Up_count = 0
        
        if Up_count >= s:
        
            return loss_meter.sum, performance_meter.avg, trajectory
        
        loss_prec = loss

In [2]:
def train_epoch(model, dataloader, loss_fn, optimizer, loss_meter, performance_meter, performance, trajectory, device): # note: I've added a generic performance to replace accuracy and the device
    for X, y in dataloader:
        # TRANSFER X AND y TO GPU IF SPECIFIED
        X = X.to(device)
        y = y.to(device)
        # ... like last time
        optimizer.zero_grad() 
        y_hat = model(X)
        loss = loss_fn(y_hat, y)
        loss = L1_reg(loss,model,0.0001)
        loss.backward()
        optimizer.step()
        acc = performance(y_hat, y)
        loss_meter.update(val=loss.item(), n=X.shape[0])
        performance_meter.update(val=acc, n=X.shape[0])

        if trajectory is not None:

            params_norm, gradients_norm = get_params_and_gradients_norm(model.named_parameters())
            trajectory["parameters"].append(params_norm)
            trajectory["gradients"].append(gradients_norm)

def train_model(model, dataloader, loss_fn, optimizer, num_epochs, alpha,s,checkpoint_loc=None, checkpoint_name="checkpoint.pt", performance=accuracy, trajectory=None, device=None): # note: I've added a generic performance to replace accuracy and an object where to store the trajectory and the device on which to run our training

    # establish device
    if device is None:
        device = "cuda:0" if torch.cuda.is_available() else "cpu"
    print(f"Training on {device}")

    # create the folder for the checkpoints (if it's not None)
    if checkpoint_loc is not None:
        os.makedirs(checkpoint_loc, exist_ok=True)
    
    model.to(device)
    model.train()
    E_opt = 10**7
    Up_count = 0 
    loss_prec = 10**7
    # epoch loop
    for epoch in range(num_epochs):
        loss_meter = AverageMeter()
        performance_meter = AverageMeter()

        train_epoch(model, dataloader, loss_fn, optimizer, loss_meter, performance_meter, performance, trajectory, device)

        print(f"Epoch {epoch+1} completed. Loss - total: {loss_meter.sum} - average: {loss_meter.avg}; Performance: {performance_meter.avg}")

        # produce checkpoint dictionary -- but only if the name and folder of the checkpoint are not None
        if checkpoint_name is not None and checkpoint_loc is not None:
            checkpoint_dict = {
                "parameters": model.state_dict(),
                "optimizer": optimizer.state_dict(),
                "epoch": epoch
            }
            torch.save(checkpoint_dict, os.path.join(checkpoint_loc, checkpoint_name))
        loss, _ = test_model(model,testloader,performance,loss_fn)
        if loss > loss_prec:
            Up_count += 1
        else :
            Up_count = 0
        if Up_count >= s:
            return loss_meter.sum, performance_meter.avg, trajectory
        loss_prec = loss
        E_opt = min(loss,E_opt)
        gl = 100*(loss/E_opt-1)
        if gl > alpha:
            return loss_meter.sum, performance_meter.avg, trajectory


    return loss_meter.sum, performance_meter.avg, trajectory

def test_model(model, dataloader, performance=accuracy, loss_fn=None, device=None):
    # establish device
    if device is None:
        device = "cuda:0" if torch.cuda.is_available() else "cpu"

    # create an AverageMeter for the loss if passed
    if loss_fn is not None:
        loss_meter = AverageMeter()
    
    performance_meter = AverageMeter()

    model.to(device)
    model.eval()
    with torch.no_grad():
        for X, y in dataloader:
            X = X.to(device)
            y = y.to(device)

            y_hat = model(X)
            loss = loss_fn(y_hat, y) if loss_fn is not None else None
            acc = performance(y_hat, y)
            if loss_fn is not None:
                loss_meter.update(loss.item(), X.shape[0])
            performance_meter.update(acc, X.shape[0])
    # get final performances
    fin_loss = loss_meter.sum if loss_fn is not None else None
    fin_perf = performance_meter.avg
    print(f"TESTING - loss {fin_loss if fin_loss is not None else '--'} - performance {fin_perf}")
    return fin_loss, fin_perf

NameError: name 'accuracy' is not defined